In [ ]:
import os
import splunklib

In [ ]:
os.getcwd()

In [2]:
from splunklib import client
import splunklib.results as results
import pandas as pd
import numpy as np
import datetime


f = open('PW.txt','r') # create a .txt file in the root folder and call it PW. only put your password in the file (no extra characters)
PW=f.read()

service = client.connect(host="100.70.36.111", port=8089, username="t928538", password=PW, app="pm_huawei_data") #Change username to own user


In [3]:
now = datetime.datetime.now()
latest_time=now.isoformat()

earliest_time_1month=(now - datetime.timedelta(days=now.day)).replace(day=now.day).isoformat()
earliest_time_1week = (now-datetime.timedelta(days=7)).isoformat()
earliest_time_1day = (now-datetime.timedelta(days=1)).isoformat()
print(latest_time)
print(earliest_time_1month)
print(earliest_time_1week)
print(earliest_time_1day)

2022-12-01T15:26:15.989115
2022-11-01T15:26:15.989115
2022-11-24T15:26:15.989115
2022-11-30T15:26:15.989115


In [4]:
site_list="AB1872,AB0010,AB0038,"

In [14]:
kpi_list="VoLTE_DCR,VoLTE_AFR"

In [15]:
start_time="2022-11-01 00:00:00"
end_time="2022-11-05 00:00:00"

start_time=datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S').isoformat()
end_time=datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S').isoformat()


In [16]:
start_time,end_time

('2022-11-01T00:00:00', '2022-11-05T00:00:00')

In [17]:
 kwargs_custom = {"earliest_time": start_time,
            "latest_time": end_time ,
            "outout_mode": "CSV",
            "count": 0}

In [18]:
def splunk_query_tocsv(query, kwargs, csvfilename):
    a = datetime.datetime.now()
    query_results=service.jobs.oneshot(query, **kwargs)
    reader=results.ResultsReader(query_results)
    print('Querying Splunk...')
    df=pd.DataFrame()
    for result in reader:
        if isinstance(result, dict):
            df=df.append(result, ignore_index=True)
    print(csvfilename + ' Saved!')
    df.to_csv(csvfilename +'.csv', index=False)
    b = datetime.datetime.now()
    print('Time Taken: ', (b-a))
    return df

In [19]:
perf_query='''
    | `PmTimeseries("1d","Coverage_SiteID","{SiteID}","Vendor,SiteID,Sector,Band,CellName","{kpi_list}")`
    '''.format(SiteID=site_list,kpi_list=kpi_list)

In [21]:
splunk_query_tocsv(query=perf_query,kwargs=kwargs_custom,csvfilename='sample00')

Querying Splunk...
sample00 Saved!
Time Taken:  0:00:48.807957


,_time,Vendor,SiteID,Sector,Band,CellName,VoLTE_DCR,VoLTE_AFR
0,2022-11-01T00:00:00.000-06:00,Huawei,AB000088,AB000088-1,1900-1,LTJAB0372479-040-1900-1-1,0,0
1,2022-11-01T00:00:00.000-06:00,Huawei,AB000088,AB000088-1,1900-1,LTJAB0372480-040-1900-1-1,0,0
2,2022-11-01T00:00:00.000-06:00,Huawei,AB000088,AB000088-1,1900-1,LTJAB0372481-040-1900-1-1,0,0
3,2022-11-01T00:00:00.000-06:00,Huawei,AB000088,AB000088-1,1900-1,LTJAB0372482-040-1900-1-1,0,0
4,2022-11-01T00:00:00.000-06:00,Huawei,AB000088,AB000088-1,1900-1,LTJAB0372483-040-1900-1-1,0,0
...,...,...,...,...,...,...,...,...
2903,2022-11-04T00:00:00.000-06:00,Samsung,AB2036,AB2036-3,2100-1,LTSAB0223351-002-2100-1-3,0,0
2904,2022-11-04T00:00:00.000-06:00,Samsung,AB2036,AB2036-3,700-1,LTSAB0223351-012-0700-1-3,1.1695906432748537,0
2905,2022-11-04T00:00:00.000-06:00,Samsung,AB2036,AB2036-3,700-2,LTSAB0223351-022-0700-2-3,0,0
2906,2022-11-04T00:00:00.000-06:00,Samsung,AB2036,AB2036-3,700-3,LTSAB0223351-032-0700-3-3,NaN,NaN
